#  Regression on Diamonds Price Dataset with SVM

The **Diamonds dataset** from Kaggle is a dataset containing information about the physical and pricing attributes of nearly 54,000 diamonds. The dataset is commonly employed in tasks like regression analysis, feature engineering, and exploratory data analysis.

We will consider a **reduced version** of the dataset, containing 4000 samples, and without categorical features.

### Key Features:
- **Carat**: The weight of the diamond.
- **Depth**: The total depth percentage (z / mean(x, y)).
- **Table**: Width of the diamond's top as a percentage of its widest point.
- **Price**: Price in US dollars.
- **X, Y, Z**: Dimensions of the diamond in mm (length, width, depth).

This dataset is useful for exploring relationships between physical attributes and pricing, and for building predictive models to estimate diamond prices based on their features.

For more information see: https://www.kaggle.com/datasets/shivam2503/diamonds.

# Overview

In the notebook you will perform a complete pipeline of machine learning - regression task. First, you will:
- split the data into training, validation, and test;
- standardize the data.

You will then be asked to learn various SVM models, in particular:
- for each of the kernels *linear*, *poly*, *rbf*, and *sigmoid*, you will learn the best model, choosing among some fixed values of the considered hyperparameters. In particular, the choice of hyperparameters must be done with **5-fold cross-validation**, as we have seen in the labs.

Then, from the models trained with the best hyperparameters selected as above, you will:
- choose the best kernel, using a validation approach (not cross-validation), and
- learn the best SVM model overall.

Furthermore, you will then be asked to estimate the generalization error of the best SVM model you report. 

At the end, just for comparison, you will also be asked to learn a standard linear regression model (with squared loss), and estimate its generalization error.

### IMPORTANT
- Note that in each of the above steps you will have to choose the appropriate split of the data (see the first bullet point above);
- The code should run without requiring modifications even if some best choice of parameters, changes; for example, you should not pass the best value of hyperparameters "manually" (i.e., passing the values as input parameters to the models). The only exception is in the TO DO titled 'ANSWER THE FOLLOWING'
- $\texttt{epsilon}$ parameter: For SVM, since the values to be predicted are all in the thousands of dollars, you will need to always set $\texttt{epsilon} = 100$
- Do not change the printing instructions (other than adding the correct variable name for your code), and do not add printing instructions!

## TO DO - INSERT YOUR NUMERO DI MATRICOLA BELOW

In [1]:
# -- put here your ID number (numero di matricola)
numero_di_matricola = 2157196 # COMPLETE

The following code loads all required packages

In [2]:
# -- import all packages needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm
from sklearn import model_selection
from sklearn import linear_model
from sklearn.model_selection import KFold
from itertools import product

The code below loads the data and remove samples with missing values. It also prints the number of samples and a brief description of our dataset.

In [3]:
# -- load the data - do not change the path below!
df = pd.read_csv('diamonds.csv', sep = ',')

# -- remove the data samples with missing values (NaN)
df = df.dropna()
# -- let's drop the column containing the id of the data
df = df.drop(columns=['Unnamed: 0'], axis=1)

In [4]:
print('Dataset shape:', df.shape)
# -- description of dataset
print(df.describe())

Dataset shape: (4000, 7)
             carat        depth       table         price            x  \
count  4000.000000  4000.000000  4000.00000   4000.000000  4000.000000   
mean      0.797945    61.776925    57.44035   3920.239250     5.735810   
std       0.462251     1.468899     2.26052   3935.292841     1.106897   
min       0.210000    52.200000    52.00000    339.000000     0.000000   
25%       0.400000    61.100000    56.00000    936.000000     4.720000   
50%       0.710000    61.900000    57.00000   2468.000000     5.710000   
75%       1.050000    62.500000    59.00000   5297.500000     6.550000   
max       3.010000    70.600000    79.00000  18730.000000     9.100000   

                 y            z  
count  4000.000000  4000.000000  
mean      5.736307     3.540002  
std       1.099129     0.691834  
min       0.000000     0.000000  
25%       4.730000     2.910000  
50%       5.730000     3.540000  
75%       6.550000     4.040000  
max       8.970000     5.670000  


In [5]:
print('First 5 samples of the dataset:\n\n', df.head(5))

First 5 samples of the dataset:

    carat  depth  table  price     x     y     z
0   0.33   61.7   55.0    564  4.43  4.46  2.74
1   1.20   62.1   57.0   5914  6.78  6.71  4.19
2   0.62   61.0   57.0   2562  5.51  5.54  3.37
3   0.34   63.1   56.0    537  4.41  4.46  2.80
4   1.20   62.5   55.0   5964  6.77  6.84  4.25


In the following cell, we convert our (pandas) dataframe into set X (containing our features) and the set Y (containing our target, i.e., the price)

In [6]:
m = df.shape[0]

# -- let's compute X and Y sets
X = df.drop(columns=['price'], axis=1)
Y = df['price']

print("Total number of samples:", m)

X = X.values
Y = Y.values

# -- print shapes
print('X shape: ', X.shape)
print('Y shape: ', Y.shape)

Total number of samples: 4000
X shape:  (4000, 6)
Y shape:  (4000,)


# Data preprocessing

## TO DO - SPLIT DATA INTO TRAINING, VALIDATION, AND TESTING, WITH THE FOLLOWING PERCENTAGES: 60%, 20%, 20%

Use the $\texttt{train\_test\_split}$ function from sklearn.model_selection to do it; in every call fix $\texttt{random\_state}$ to your numero_di_matricola. 
At the end, you should store the data in the following variables:
- X_train, Y_train: training data;
- X_val, Y_val: validation data;
- X_train_val, Y_train_val: training and validation data;
- X_test, Y_test: test data.

The code then prints the number of samples in X_train, X_val, X_train_val, and X_test

**IMPORTANT:**
- first split the data into training+validation and test; the first part of the data in output from $\texttt{train\_test\_split}$ must correspond to the training+validation;
- then split training+validation into training and validation; the first part of the data in output from $\texttt{train\_test\_split}$ must correspond to the training


In [7]:
# -- split the data into training + validation and test
m_train_val = int(4/5 * m)
m_test = m - m_train_val

X_train_val, X_test , Y_train_val, Y_test = train_test_split(X, Y , test_size= m_test, random_state=numero_di_matricola) 

# -- split the training + validation data into training and validation
m_train = int(3/4 * m_train_val)
m_val = m_train_val - m_train

X_train , X_val , Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size= m_val, random_state = numero_di_matricola)


print("Training size:", X_train.shape[0])
print("Validation size:", X_val.shape[0])
print("Training and validation size:", X_train_val.shape[0])
print("Test size:", X_test.shape[0])

Training size: 2400
Validation size: 800
Training and validation size: 3200
Test size: 800


## TO DO - STANDARDIZE THE DATA

Standardize the data using the $\texttt{preprocessing.StandardScaler}$ from scikit learn.

If V is the name of the variable storing part of the data, the corresponding standardized version should be stored in V_scaled. For example, the scaled version of X_train should be stored in X_train_scaled.

In [8]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_train_val_scaled = scaler.transform(X_train_val)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# SVM models: learning the best model for each kernel

The following function, i.e., $\texttt{k\_fold\_cross\_validation}$, will perform $k$-fold cross validation (with $k$ = 5 by default). Look carefully at the signature of the below function: you have in input some sets X and Y, the default number of folds, and a length-variable keyword argumens, with which the SVM model will be trained in the cross-validation phase. If you are not familiar with the notation, look at kwargs in Python documentation.

In the first lines of the below function, the unpacked parameters (i.e., input parameter $\texttt{param\_grid}$) are converted into a python list by means of cartesian product. The resulting list (i.e., $\texttt{param\_list}$) will be the one for which you need to iterate over and perform $k$-fold cross-validation, using $\texttt{KFold}$ object frmo scikit-learn.

At the end, note that you need to return $\texttt{best\_param}$, that is the best set of parameters you found with the cross-validation procedure. 

In [9]:
def k_fold_cross_validation(X, Y, num_folds = 5, **param_grid):

    # -- grid of hyperparams into list
    param_keys = list(param_grid.keys())
    param_values = list(param_grid.values())
    
    # Generate Cartesian product of values
    combinations = product(*param_values)
    
    # Create a list of dictionaries from combinations
    param_list = [dict(zip(param_keys, combination)) for combination in combinations]

    err_train_kfold = np.zeros(len(param_list))
    err_val_kfold = np.zeros(len(param_list))
    
    kf = KFold(n_splits = num_folds) 

    for i,params in enumerate(param_list):

        model_svm = svm.SVR(**params)

        for train_index, val_index in kf.split(X):

            X_train_kfold, X_val_kfold = X[train_index], X[val_index]
            Y_train_kfold, Y_val_kfold = Y[train_index], Y[val_index]

            scalerKF = preprocessing.StandardScaler()
            scalerKF.fit(X_train_kfold)
            X_train_kfold_scaled = scalerKF.transform(X_train_kfold)
            X_val_kfold_scaled = scalerKF.transform(X_val_kfold) 

            model_svm.fit(X_train_kfold_scaled, Y_train_kfold)

            err_train_kfold[i] += (1.0 - model_svm.score(X_train_kfold_scaled,Y_train_kfold))
            err_val_kfold[i] += (1.0 - model_svm.score(X_val_kfold_scaled, Y_val_kfold))

        err_train_kfold[i] /= num_folds 
        err_val_kfold[i] /= num_folds

    best_params = param_list[np.argmin(err_val_kfold)] 

    return best_params



## TO DO - CHOOSE THE BEST HYPERPARAMETERS FOR LINEAR KERNEL

For the SVM, consider $\texttt{svm.SVR}$ class. We will begin by training the SVM with linear kernel. For the latter, consider the following hyperparameters and their values:

- $C: [0.1, 1, 10, 100, 1000]$

Remember that both the $\texttt{kernel}$ type and the value of $\texttt{epsilon}$ are considered as parameters to pass to the above method. Leave all other input parameters to default. 

Find the best value of the hyperparameters using 5-fold cross validation. Use the function defined above to perform the cross-validation.

Print the best value of the hyperparameters.

In [10]:
print("\nLinear SVM:")

param_grid1 = {
    'C': [0.1, 1, 10 , 100, 1000],
    'kernel':["linear"],
    'epsilon': [100]
}

opt_param1 = k_fold_cross_validation(X_train_scaled,Y_train, **param_grid1)

print("Best value for hyperparameters: ", opt_param1)


Linear SVM:
Best value for hyperparameters:  {'C': 1000, 'kernel': 'linear', 'epsilon': 100}


## TO DO - LEARN A MODEL WITH LINEAR KERNEL AND BEST CHOICE OF HYPERPARAMETERS

This model will be compared with the best models with other kernels using validation (not cross validation).

DO NOT PASS PARAMETERS BY HARD-CODING THEM IN THE CODE.

Print the **training score** (that is, $R^2$ coefficient) of the best model, trained with the best parameter find from the above cell.

In [11]:
model_svm1 = svm.SVR(**opt_param1)

model_svm1.fit(X_train_scaled,Y_train)
training_score =  model_svm1.score(X_train_scaled,Y_train)

print("Training score:", training_score)

Training score: 0.8478034801736155


## TO DO - CHOOSE THE BEST HYPERPARAMETERS FOR POLY KERNEL

Now, let's consider $\texttt{svm.SVR}$ with polynomial kernel. Consider the following hyperparameters and their values:
- $C: [0.1, 1, 10, 100, 1000]$
- $degree: [2, 3, 4]$

Leave all other input parameters to default. 

Find the best value of the hyperparameters using 5-fold cross validation. Use the function defined above to perform the cross-validation.

Print the best value of the hyperparameters.

In [12]:
print("\nPoly SVM")

param_grid2 = {
    'C': [0.1, 1, 10 , 100, 1000],
    'kernel':["poly"],
    'degree':[2,3,4],
    'epsilon': [100]
}

opt_param2 = k_fold_cross_validation(X_train_scaled,Y_train, **param_grid2)

print("Best value for hyperparameters: ", opt_param2)


Poly SVM
Best value for hyperparameters:  {'C': 10, 'kernel': 'poly', 'degree': 3, 'epsilon': 100}


## TO DO - LEARN A MODEL WITH POLY KERNEL AND BEST CHOICE OF HYPERPARAMETERS

This model will be compared with the best models with other kernels using validation (not cross validation).

DO NOT PASS PARAMETERS BY HARD-CODING THEM IN THE CODE.

Print the **training score** (that is, $R^2$ coefficient) of the best model, trained with the best parameter find from the above cell.

In [13]:
model_svm2 = svm.SVR(**opt_param2)

model_svm2.fit(X_train_scaled,Y_train)
training_score = model_svm2.score(X_train_scaled,Y_train)

print("Training score:",training_score)

Training score: 0.4588069990934117


## TO DO - CHOOSE THE BEST HYPERPARAMETERS FOR RBF KERNEL

Consider $\texttt{svm.SVR}$ with RBF kernel. Consider the following hyperparameters and their values:
- $C: [0.1, 1, 10, 100, 1000]$
- $gamma: [0.01, 0.03, 0.04, 0.05]$

Leave all other input parameters to default. 

Find the best value of the hyperparameters using 5-fold cross validation. Use the function defined above to perform the cross-validation.

Print the best value of the hyperparameters.

In [14]:
print("\nRBF SVM")

param_grid3 = {
    'C': [0.1, 1, 10 , 100, 1000],
    'kernel':["rbf"],
    'gamma':[0.01, 0.03, 0.04, 0.05],
    'epsilon': [100]
}

opt_param3 = k_fold_cross_validation(X_train_scaled,Y_train, **param_grid3)

print("Best value for hyperparameters: ", opt_param3)


RBF SVM
Best value for hyperparameters:  {'C': 1000, 'kernel': 'rbf', 'gamma': 0.05, 'epsilon': 100}


## TO DO - LEARN A MODEL WITH RBF KERNEL AND BEST CHOICE OF HYPERPARAMETERS

This model will be compared with the best models with other kernels using validation (not cross validation).

DO NOT PASS PARAMETERS BY HARD-CODING THEM IN THE CODE.

Print the **training score** (that is, $R^2$ coefficient) of the best model, trained with the best parameter find from the above cell.

In [15]:
model_svm3 = svm.SVR(**opt_param3)

model_svm3.fit(X_train_scaled,Y_train)
training_score = model_svm3.score(X_train_scaled,Y_train)

print("Training score:",training_score)

Training score: 0.876498370098041


## TO DO - CHOOSE THE BEST HYPERPARAMETERS FOR SIGMOID KERNEL

Consider $\texttt{svm.SVR}$ with sigmoid kernel. Consider the following hyperparameters and their values:
- $C: [0.1, 1, 10, 100, 1000]$
- $gamma: [0.01, 0.05, 0.1]$
- $coef0: [0, 1]$

Leave all other input parameters to default. 

Find the best value of the hyperparameters using 5-fold cross validation. Use the function defined above to perform the cross-validation.

Print the best value of the hyperparameters.

In [16]:
print("\nSigmoid SVM")

param_grid4 = {
    'C': [0.1, 1, 10 , 100, 1000],
    'kernel':["sigmoid"],
    'gamma':[0.01, 0.05, 0.1],
    'coef0': [0,1],
    'epsilon': [100]
}

opt_param4 = k_fold_cross_validation(X_train_scaled,Y_train, **param_grid4)

print("Best value for hyperparameters: ", opt_param4)


Sigmoid SVM
Best value for hyperparameters:  {'C': 1000, 'kernel': 'sigmoid', 'gamma': 0.01, 'coef0': 0, 'epsilon': 100}


## TO DO - LEARN A MODEL WITH SIGMOID KERNEL AND BEST CHOICE OF HYPERPARAMETERS

This model will be compared with the best models with other kernels using validation (not cross validation).

DO NOT PASS PARAMETERS BY HARD-CODING THEM IN THE CODE.

Print the **training score** (that is, $R^2$ coefficient) of the best model, trained with the best parameter find from the above cell.

In [17]:
model_svm4 = svm.SVR(**opt_param4)

model_svm4.fit(X_train_scaled,Y_train)
training_score = model_svm4.score(X_train_scaled,Y_train)

print("Training score:",training_score)

Training score: 0.7932289634503789


## TO DO - USE VALIDATION TO CHOOSE THE BEST MODEL AMONG THE ONES LEARNED FOR THE VARIOUS KERNELS

Use validation to choose the best model among the four ones (one for each kernel) you have learned above.

Print, following exactly the order described here, with 1 value for each line:
- the validation score of SVM with linear kernel (the template below does not include such print)
- the validation score of SVM with polynomial kernel (the template below does not include such print)
- the validation score of SVM with rbf kernel (the template below does not include such print)
- the validation score of SVM with sigmoid kernel (the template below does not include such print)
- the best kernel (e.g., sigmoid) 
- the validation score of the best kernel 

For the first 4 prints, use the format: "*kernel* validation score: ". For example, for linear kernel "linear validation score: ", for rbf "rbf validation score: "

In [18]:
print("\nVALIDATION TO CHOOSE SVM KERNEL:")

models = []
models_val_score = []

print("Linear validation score:", model_svm1.score(X_val_scaled,Y_val))
print("Poly validation score:",model_svm2.score(X_val_scaled,Y_val))
print("Rbf validation score:",model_svm3.score(X_val_scaled,Y_val))
print("Sigmoid validation score:", model_svm4.score(X_val_scaled,Y_val))

models.append(model_svm1)
models.append(model_svm2)
models.append(model_svm3)
models.append(model_svm4)

models_val_score.append(model_svm1.score(X_val_scaled,Y_val))
models_val_score.append(model_svm2.score(X_val_scaled,Y_val))
models_val_score.append(model_svm3.score(X_val_scaled,Y_val))
models_val_score.append(model_svm4.score(X_val_scaled,Y_val))

best_model = models[np.argmax(models_val_score)] 

print("\n---\nBest kernel: ", best_model.kernel)
print("Validation score of best kernel: ", models_val_score[np.argmax(models_val_score)])


VALIDATION TO CHOOSE SVM KERNEL:
Linear validation score: 0.8343302126682242
Poly validation score: 0.5918955226383735
Rbf validation score: 0.8534406943343721
Sigmoid validation score: 0.7788184820223727

---
Best kernel:  rbf
Validation score of best kernel:  0.8534406943343721


## TO DO - LEARN THE FINAL MODEL FOR WHICH YOU WANT TO ESTIMATE THE GENERALIZATION SCORE

Learn the final model (i.e., the one you would use to make predictions about future data).

Print the **final model hyperparameters** and the **score** of the model on the data used to learn it.

In [19]:
print("\nBEST MODEL:")

best_model.fit(X_train_val_scaled, Y_train_val)

print("Best model hyperparameters:",best_model.get_params())
print("Score of the best model on the data used to learn it: ", best_model.score(X_train_val_scaled,Y_train_val))


BEST MODEL:
Best model hyperparameters: {'C': 1000, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 100, 'gamma': 0.05, 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Score of the best model on the data used to learn it:  0.8715903814366934


## TO DO - PRINT THE ESTIMATE  OF THE GENERALIZATION SCORE FOR THE FINAL MODEL

Print the estimate of the generalization **score** for the final model. The generalization "score" is the score computed on the data used to estimate the generalization error.

In [20]:
print("\nGENERALIZATION SCORE BEST MODEL:")
print("Estimate of the generalization score for best SVM model: ", best_model.score(X_test_scaled,Y_test))


GENERALIZATION SCORE BEST MODEL:
Estimate of the generalization score for best SVM model:  0.8388150442722601


## TO DO - ANSWER THE FOLLOWING

Print the **training score** (score on data used to train the model) and the **generalization score** (score on data used to assess generalization) of the final SVM model THAT YOU OBTAIN WHEN YOU RUN THE CODE, one per line, printing the smallest one first. 

NOTE: THE VALUES HERE SHOULD BE HARDCODED.

Print you answer (YES/NO) to the following question: does the relation (i.e., smaller, larger) between the training score and the generalization score agree with the theory?

Print your motivation for the YES/NO answer above, using at most 500 characters.

In [21]:
print("\nANSWER")
print("YES")

# -- note that you may have to invert the order of the following 2 lines, print the smallest 1 first
train_score = best_model.score(X_train_val_scaled,Y_train_val)
gen_score = best_model.score(X_test_scaled,Y_test)


print("Generalization score: ", 0.8388150442722601) 
print("Training score: ", 0.8715903814366934)

    
# -- the following is a string with you answer
motivation = """\nThis result aligns with theoretical expectations, as the generalization error 
(an approximation of the true error) should closely emulate the training error while minimizing
the overall error. The best model aims to perform well on new data, even if its distribution or noise
differs from the training samples. A slightly higher generalization error on a test set is acceptable
and typically indicates a slight overfitting, which is natural when learning from finite training data."""

print(motivation)


ANSWER
YES
Generalization score:  0.8388150442722601
Training score:  0.8715903814366934

This result aligns with theoretical expectations, as the generalization error 
(an approximation of the true error) should closely emulate the training error while minimizing
the overall error. The best model aims to perform well on new data, even if its distribution or noise
differs from the training samples. A slightly higher generalization error on a test set is acceptable
and typically indicates a slight overfitting, which is natural when learning from finite training data.


## TO DO: LEARN A STANDARD LINEAR MODEL
Learn a standard linear model using scikit learn.

Print the **score** of the model on the data used to learn it.

Print the **generalization score** of the model.

In [22]:
print("\nLR MODEL")


model_linear = linear_model.LinearRegression()
model_linear.fit(X_train_val_scaled, Y_train_val)


print("Score of LR model on data used to learn it: ", model_linear.score(X_train_val_scaled,Y_train_val))
print("Generalization score of LR model: ", model_linear.score(X_test_scaled,Y_test))


LR MODEL
Score of LR model on data used to learn it:  0.8554836530540636
Generalization score of LR model:  0.8394131615067224
